'''
v1: Scienceon, DBPIA, NTIS AuthorPapers 접근하여 Rawdata에서 필터링된 papers를 추가
v2: Answer 변경 및 f_id 수정 중
v3: time, tqdm 추가하여 속도 확인
v4: Scienceon, DBPIA에 mng로 검색 추가
v5: DBPIA 삭제하고 KCI 추가 & Rawdata 먼저 접급으로 변경, answer check 추가
v5.1: 대학테이블, 정답비교 코드 추가
v6: nFilter, pFilter 조건 비교 함수 추가(ntis_filter, paper_filter)
v6.1: 대학테이블, 정답비교 코드 추가, sites => site, Answer['inst'] 추가
v7: Autor 접근 삭제, Rawdata에서 소속 이름 가져오기 추가, inst값 없는 문제 해결, A_id 나눠들어가는 문제 해결
'''

In [ ]:
from pymongo import MongoClient
import itertools
import numpy as np
import jaro
import re

client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')
filter_info = client['PUBLIC']['FilterInfo'] #필터접근

f_id = 0 #input
keyid = 672 #keyid

fid_key_query = filter_info.find_one({ '$and': [{ 'fId': f_id }, { 'keyId': keyid }]}) #f_id serach
ninst = []
nrsc = []
nfund = []
nyear = []
pinst = []
pyear = []
pjournal = []
plang = []

if  fid_key_query != None: #f_id check
    for key in fid_key_query.keys() :
        if key == 'nFilter':
            ninst = fid_key_query[key]['inst'] #소속
            nrsc = fid_key_query[key]['rsc'] #공동저자수
            nfund = fid_key_query[key]['fund'] #과제수주비
            nyear = fid_key_query[key]['year'] #연도

        elif key == 'pFilter' :
            pinst = fid_key_query[key]['inst']
            pyear = fid_key_query[key]['year']
            pjournal = fid_key_query[key]['journal']
            plang = fid_key_query[key]['lang']

scion_aut = client['SCIENCEON']['Author']
ntis_aut = client['NTIS']['Author']
kci_aut = client['KCI']['Author']

scion_raw = client['SCIENCEON']['Rawdata']
ntis_raw = client['NTIS']['Rawdata']
kci_raw = client['KCI']['Rawdata']

scion_key_query = scion_raw.find({ 'keyId' : keyid })
ntis_key_query = ntis_raw.find({ 'keyId' : keyid })
kci_key_query = kci_raw.find({ 'keyId' : keyid })

auts = [scion_aut, ntis_aut, kci_aut] #Author
key_querys = [scion_key_query, ntis_key_query, kci_key_query] #Rawdata
id_domestic = client['ID']['Domestic'] #Domestic

mng_id = [] # Author id
paper = []

Answer_dict = {} # Answer result
site = ['SCIENCEON', 'NTIS', 'KCI']
fund = [0, 50000000, 100000000, 300000000, 500000000, 1000000000 ]
rsc = [0, 10, 30, 50, 100]

def simple_filter(value, filters) :
    if value in filters or filters == []:
        return True
    return False
        
def complex_filter(value, filters, base) :
    if filters == [] :
        return True

    for j in range(len(filters)):
        if base[filters[j]] <= float(value) < base[filters[j]+1]:
            return True
    return False

for i in range(len(key_querys)):
    mng_dict = {}
    for key_query in key_querys[i]: #rawdata(magid, paper) insert
        
        if site[i] == 'NTIS' :
            ori_inst = key_query['originalName']
            ntis_rsc = int(key_query['cntRscMan']) + int(key_query['cntRscWom'])
            ntis_fund = key_query['totalFund']
            ntis_year = key_query['prdStart'][:4]
            exi_inst = key_query['ldAgency']
            mng_name =  key_query['mng']
            mng_id = key_query['mngId']
            paper = [key_query['_id']]

            if simple_filter(ori_inst, ninst) and simple_filter(ntis_year, nyear) and complex_filter(ntis_fund, nfund, fund) and complex_filter(ntis_rsc, nrsc, rsc):
                if mng_id not in mng_dict:
                    mng_dict[mng_id] = {'name' : mng_name, 'inst' : exi_inst, 'papers' : [], 'oriInst' : ori_inst}
                mng_dict[mng_id]['papers'].append(paper)
                
        else:
            paper_year =  key_query['issue_year'][:4]
            paper_journal = key_query['journal']
            ori_inst = key_query['originalName'].split(';')[-2]
            paper_lang = key_query['issue_lang']
            exi_inst = key_query['author_inst'].split(';')[-2]
            mng_name = key_query['author'].split(';')[-2]
            mng_id = key_query['mngId']
            paper = [key_query['_id']]

            if simple_filter(paper_year, pyear) and simple_filter(paper_journal, pjournal) and simple_filter(ori_inst, pinst) and simple_filter(paper_lang, plang):
                if mng_id not in mng_dict:
                    mng_dict[mng_id] = {'name' : mng_name, 'inst' : exi_inst, 'papers' : [], 'oriInst' : ori_inst}
                mng_dict[mng_id]['papers'].append(paper)

    for mng_one in mng_dict :
        oriinst = mng_dict[mng_one]['oriInst']
        exiinst = mng_dict[mng_one]['inst']
        mng_name = mng_dict[mng_one]['name']
        paper = mng_dict[mng_one]['papers']

        Answer = {'fid': f_id, 'keyId': keyid, 'name' : mng_name , 'inst': oriinst, site[i] : {'inst' :exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst} }
        
        if mng_name not in Answer_dict and mng_name+'_0' not in Answer_dict : #동명이인이 없을 때
            Answer_dict[mng_name] = Answer
        else :
            count = 0
            flag = True
            while flag :
                temp = None 
                tempName = mng_name
                
                if tempName in Answer_dict : # 이름 으로만 key가ㅣ 존재         
                    temp = Answer_dict[tempName]
                    flag = False
                else :
                    tempName = mng_name+'_'+str(count) # 이름 + 숫자로 key가ㅣ 존재
                    if tempName not in Answer_dict :
                        flag = False 
                        break
                    temp = Answer_dict[tempName]
                        
                for key in temp.keys() : # 사이트 돌면서
                    if key != 'name' and key != 'keyId' and key != 'fid' and key != 'inst': 
                        src = ""
                        tgt = ""

                        if len(exiinst) >= len(temp[key]['inst']):
                            src = temp[key]['inst']
                            tgt = exiinst

                        elif len(exiinst) < len(temp[key]['inst']):
                            src = exiinst
                            tgt = temp[key]['inst']

                        if key == site[i] :# 사이트가 동일할때
                            if temp[key]['inst'] == exiinst or (src != "" and src in tgt) :  # 소속 같을때
                                Answer_dict[tempName][site[i]]['A_id'].extend([mng_one])
                                Answer_dict[tempName][site[i]]['papers'].extend(paper)
                                flag = False
                                break

                            elif mng_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[mng_name+'_'+str(count+1)] = Answer

                                if tempName == mng_name:
                                    Answer_dict[mng_name+'_0'] = temp
                                    del Answer_dict[mng_name]

                        else :# 사이트가 다를때 
                            if temp[key]['inst'] == exiinst  or (src != "" and src in tgt):  # 소속 같을때
                                Answer_dict[tempName][site[i]] =  {'inst' : exiinst, 'A_id': [mng_one], 'papers' : paper, 'oriInst' : oriinst}
                                if '대학교' in Answer_dict[tempName][site[i]]['oriInst'] and '대학교' not in Answer_dict[tempName]['inst']:
                                    Answer_dict[tempName]['inst'] = Answer_dict[tempName][site[i]]['oriInst']
                                flag = False
                                break
                            
                            elif mng_name+'_'+str(count+1) not in Answer_dict : #소속이 다를 때
                                Answer_dict[mng_name+'_'+str(count+1)] = Answer

                                if tempName == mng_name:
                                    Answer_dict[mng_name+'_0'] = temp
                                    del Answer_dict[mng_name]

                count += 1
# print(sorted(Answer_dict.items()))
#print(len(all_name))

In [35]:
def isEnglishOrKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    return "k" if k_count>1 else "e"

def check_college(univ0):
    branch_set = ['성균관대학교', '건국대학교', '한양대학교']
    univName = client['PUBLIC']['CollegeName']
    univ1 = re.sub("산학협력단|병원","",univ0)
    univ2 = re.sub("대학교","대학교 ",univ1)

    try:
        if isEnglishOrKorean(univ0) == 'e':
            univ0 = univ0.upper()
            univ0 = univ0.replace('.', ',')
            univ = univ0.split(', ')
        else:
            univ = univ2.replace(",", "").split()
            univ = list(set(univ))   
            
        for uni in univ:
            if uni in branch_set:
                if ("ERICA" or "에리카") in univ0:
                    univ[univ.index("한양대학교")] = "한양대학교(ERICA캠퍼스)"
                elif ("글로컬" or "GLOCAL") in univ0:
                    univ[univ.index("건국대학교")] = "건국대학교 GLOCAL(글로컬)캠퍼스"
                elif "자연과학캠퍼스" in univ0:
                    univ[univ.index("성균관대학교")] = "성균관대학교(자연과학캠퍼스)"

        univs = '{"$or": ['
        for u in range(len(univ)):
            if univ[-1] == univ[u]:
                univs += '{"inputName": "' + univ[u] + '"}'
            else:
                univs += '{"inputName": "' + univ[u] + '"}, '
        univs += ']}'

        univ_query = univName.find_one(eval(univs))

        if univ_query is None:
            print("Search inst None")
            return univ0, False
        else:
            #print("rawInput:[",univ0,"]","queryOutput:" ,univ_query['originalName'])
            return univ_query['originalName'], True 
        
    except SyntaxError as e:
        print(e)
        print(univ0)
        return univ0, False

def filter(site, rawdata):
    if site == 'NTIS' :
        coauthor = rawdata['rsc'].split(";")
        year = int(rawdata['prdStart'][:4])
        keyword = rawdata['koKeyword'].split(",")
        journal = ""
        conference = ""
        title = ""

    else :
        coauthor = rawdata['author'].split(";")[1:-1]
        year = int(rawdata['issue_year'][:4])
        
        if rawdata['paper_keyword'] == [] or rawdata['paper_keyword'] == 'None':
            keyword = []
        elif len(rawdata['paper_keyword']) > 1:
            for i in range(0, len(rawdata['paper_keyword'])):
                keyword = []
                keyword.append(rawdata['paper_keyword'][i].replace(" ", "").split("."))            
        else:
            keyword = rawdata['paper_keyword'].replace(" ", "").split(".")

        journal = rawdata['journal']
        conference = rawdata['issue_inst']
        title = rawdata['title']

    return coauthor, year, keyword, journal, conference, title

def Secondary_filter(name, site1, inst1, raw_one1, site2, inst2, raw_one2):
    inst = 0
    weight = 0
    joc = 0
    coauthor1, year1, keyword1, journal1, conference1, title1 = filter(site1, raw_one1)
    coauthor2, year2, keyword2, journal2, conference2, title2 = filter(site2, raw_one2)

    if inst1 == inst2:
        inst = 1
    else:
        inst = jaro.jaro_winkler_metric(inst1, inst2)

    if name in coauthor1:
        coauthor1.remove(name)

    if name in coauthor2:
        coauthor2.remove(name)
    
    co_author_count = len([i for i in coauthor1 if i in coauthor2])

    if site1 != 'NTIS' and site2 != 'NTIS' :
        if title1 == title2 or inst >= 0.8: #or mng1 == mng2:
            weight = 4
            return weight

        else:
            joc = 1 if journal1 == journal2 and conference1 == conference2 else 0
    else:
        if inst >= 0.8:
            weight = 3
            return weight
            
    yop = -(2*(abs(year1-year2)/10)-1)
            
    if len(coauthor1) == 0 or len(coauthor2) == 0:
        co_author_ratio = 0
    elif len(coauthor1) < len(coauthor2):
        co_author_ratio = co_author_count/len(coauthor1)
    else:
        co_author_ratio = co_author_count/len(coauthor2)
    
    if co_author_ratio == 1:
        co_authorship = 1
    else:
        co_authorship = (1 - np.exp(-co_author_count))/2 + (co_author_ratio/2)
    keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

    #print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

    weight = joc + yop + co_authorship + keyword

    return weight

In [38]:
raw_dbs = {'NTIS' : ntis_raw, 'SCIENCEON' : scion_raw, 'KCI' : kci_raw}

def getRaw(name):
    if 'raws' not in Answer_dict[name]:
        raws = []
        for site_one in site:
            if site_one in Answer_dict[name]:
                for c in raw_dbs[site_one].find({"_id": {"$in": Answer_dict[name][site_one]['papers']}}):
                    c['site'] = site_one
                    raws.append(c)
        
        Answer_dict[name]['raws'] = raws

processedList = []
deleteList = []

for Answer_one in Answer_dict :
    if '_' in Answer_one :
        name = Answer_one.split("_")
        if name[0] in processedList :
            continue
        preprocessedList = []
        c = 0
        while True :
            pname = name[0]+"_"+str(c)
            if pname in Answer_dict :            
                preprocessedList.append(pname)
                getRaw(pname)
                c += 1
            else :
                break
        
        processedList.append(name[0])
        flag = True
        while flag :
            flag = False
            pairs =list(itertools.combinations(preprocessedList, 2))
            
            for pair in pairs:
                pair = list(pair)

                raws1 = Answer_dict[pair[0]]['raws']
                raws2 = Answer_dict[pair[1]]['raws']
                
                for ra1, ra2 in zip(raws1, raws2):
                    site1 = ra1['site']
                    site2 = ra2['site']
                    inst1 = Answer_dict[pair[0]][site1]['oriInst']
                    inst2 = Answer_dict[pair[1]][site2]['oriInst']

                    if Secondary_filter(name[0], site1, inst1, ra1, site2, inst2, ra2) >= 3:
                        deleteList.append(pair[1])
                        for site_one in site:
                            if site_one in Answer_dict[pair[1]]:
                                if site_one in Answer_dict[pair[0]].keys() :                            
                                    Answer_dict[pair[0]][site_one]['mng_id'].extend(Answer_dict[pair[1]][site_one]['mng_id'])
                                    Answer_dict[pair[0]][site_one]['papers'].extend(Answer_dict[pair[1]][site_one]['papers'])
                                    Answer_dict[pair[0]]['raws'].extend(Answer_dict[pair[1]]['raws'])
                                else:
                                    Answer_dict[pair[0]][site_one] = Answer_dict[pair[1]][site_one]
                        flag = True
                        preprocessedList.remove(pair[1])
                        break
                if flag :
                    break
            
for d in deleteList:
    del Answer_dict[d]

for d in Answer_dict : 
    if 'raws' in Answer_dict[d] :
        del Answer_dict[d]['raws']

# print(Answer_dict)
# id_domestic.insert_many(Answer_dict.values()) #mongodb 추가

{'유현규': {'fid': 0, 'keyId': 672, 'name': '유현규', 'inst': 'SW-SoC융합연구소', 'SCIENCEON': {'inst': 'SW-SoC융합연구소', 'A_id': ['s1002924'], 'papers': [[ObjectId('61dceae78e06f1dc11b59740')]], 'oriInst': 'SW-SoC융합연구소'}}, '송인채': {'fid': 0, 'keyId': 672, 'name': '송인채', 'inst': '숭실대학교', 'SCIENCEON': {'inst': '숭실대학교 전자공학과, 현대오트론 차량반도체개발센터', 'A_id': ['s1044692'], 'papers': [[ObjectId('61dceae98e06f1dc11b59742')]], 'oriInst': '숭실대학교'}}, '한혁수': {'fid': 0, 'keyId': 672, 'name': '한혁수', 'inst': '상명대학교', 'SCIENCEON': {'inst': '상명대학교 컴퓨터과학부', 'A_id': ['s709051'], 'papers': [[ObjectId('61dceaee8e06f1dc11b59744')]], 'oriInst': '상명대학교'}, 'KCI': {'inst': '상명대학교', 'A_id': ['s54750'], 'papers': [[ObjectId('61dceafe5e69900f1fb58d16')]], 'oriInst': '상명대학교'}}, '김동원': {'fid': 0, 'keyId': 672, 'name': '김동원', 'inst': '충북도립대학교', 'SCIENCEON': {'inst': '충북도립대학 반도체전자과', 'A_id': ['s286086'], 'papers': [[ObjectId('61dceaee8e06f1dc11b59746')]], 'oriInst': '충북도립대학교'}}, '엄낙웅': {'fid': 0, 'keyId': 672, 'name': '엄낙웅', 'inst': '시스템

In [10]:
import json
Answer_dict = Answer_dict

answer = None
numCor = 0
numErr = 0
numInteErr = 0
answerCounter = {}

def ansCheck(result, name) :
    global answer, numCor, numErr, answerCounter, numInteErr
    flag = True

    numK = len(result.keys())    
    if numK-4 != len(answer[name].keys()) :
        flag = False
    else :
        for site in result.keys() :        
            if site != 'name' and site != 'fid' and site != 'keyId' and site != 'inst':
                if site not in answer[name].keys() :
                    flag = False
                else: 
                    if numK > 5:
                        if answer[name][site] != result[site]['inst'] :
                            flag = False
                    else :
                        if answer[name][site] != result[site]['inst'] and  answer[name][site] != result[site]['inst'].split(' ')[0]:
                            flag = False
    if flag :
        numCor += 1
        answerCounter[name] += 1
        #print(f"Correct \n - Answer : {answer[name]}\n - Result : {result}")
    else :
       
        if len(answer[name].keys()) > 1 :
            numInteErr += 1
            print(f"IIIIIIIIIIIIIIIIIIIIIIIIIIII Not Correct \n - Answer : {answer[name]}\n - Result : {result}")
        else :
            pass
       #    print(f"Not Correct \n - Answer : {answer[name]}\n - Result : {result}")
        numErr += 1
    return flag

with open(f'answer{keyid}.json', 'r',encoding='utf-8-sig') as a_json :

    answer = json.load(a_json)
    numAns = print(len(answer))
    for key in answer.keys() :
        answerCounter[key] = 0

    for answer_one in Answer_dict:
        test = answer_one
        
        name = Answer_dict[test]['name'].split('_')[0]
        
        if name in answer :
            ansCheck(Answer_dict[test], name)

        else :
            count = 0
            # if name+str(count) not in answer :
            #     print(name)
            while name+str(count) in answer :
                # if name+count in answer :
             #   print(f"Check {name+str(count)}")
                if ansCheck(Answer_dict[test], name+str(count)) :
                    break
                count += 1

    print(f"num코렉트 {numCor}")
    print(f"numEror {numErr}")
    print(f"numInteEror {numInteErr}")
    totalError = 0
    totalInteError1 = 0
    totalInteError2 = 0
    answercount = 0
    totalInte = 0

    for key in answerCounter.keys() :
        answercount += 1
        if answerCounter[key] == 0 :
            totalError += 1
            print("answer=",key, answer[key])
            if len(answer[key].keys()) > 1 :
                totalInteError1 += 1

        else:
            if len(answer[key].keys()) > 1 :
                totalInteError2 += 1

                
                #print("answer=",key, answer[key])
                '''
                if key not in Answer_dict :
                    count = 0
                    
                    while key+str(count) in Answer_dict :
                        # if name+count in answer :
                        print("result=", Answer_dict[ key+str(count)])                        
                        count += 1
                else :    
                    print("result=", Answer_dict[key])
                '''
            #print(key)|
            #print(answer[key])

    print("토탈앤써", answercount) 
    print("토탈인터그레이션", totalInteError1 + totalInteError2)
    print("--------------------------------")
    print("토탈에러", totalError) 
    print("토탈인터그레이션에러", totalInteError1)
    
    print(keyid)    

#print(Answer_dict)


FileNotFoundError: [Errno 2] No such file or directory: 'answer672.json'

In [5]:
if '대학교' in '충북대학교 에베베학부 홀로로확과':
    print('check')

a = jaro.jaro_winkler_metric('성균관대학교', '성균관대학교(자연과학캠퍼스)')
print(a)

check
0.88
